# mujoco 基础知识学习
根据 mujoco 官方提供的tutorial 来学习
官方的连接如下[mujoco tutorial](https://colab.research.google.com/github/google-deepmind/mujoco/blob/main/python/tutorial.ipynb#scrollTo=MpkYHwCqk7W-)


In [1]:
# 基础的一些库导入
import mujoco
import time
import itertools
import numpy as np
import mediapy as media
import matplotlib.pyplot as plt

## 从定义和加载一个简单的模型开始

In [3]:
xml = """
<mujoco>
  <worldbody>
    <geom name="red_box" type="box" size=".2 .2 .2" rgba="1 0 0 1"/>
    <geom name="green_sphere" pos=".2 .2 .2" size=".1" rgba="0 1 0 1"/>
  </worldbody>
</mujoco>
"""
model = mujoco.MjModel.from_xml_string(xml)

这份xml是 MuJoCo 的 MJCF 格式，是一种基于XML 的模型语言
+ 唯一需要的element is ```<mujoco>```,最小的合法MJCF 模型就是```<mujoco/>```， 是一个空的模型
+ 所有的物理元素都包含在```<worldbody>```这个里面。它是最顶层的body
+ 这个```from_xml_string()``` 方法调用了模型的编译器，然后创建了一个```mjModel``` 实例

## mjModel
```mjModel``` 包含了模型的描述，是不会随着时间改变的一些量。
例如```ngeom``` 是场景中的几何体的数量， ```geom_rgba``` 是每个几何体的颜色

In [5]:
print(model.ngeom)
model.geom_rgba

2


array([[1., 0., 0., 1.],
       [0., 1., 0., 1.]], dtype=float32)